In [76]:
import torch
import random
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import torch_geometric as pyg
import torch_geometric.data as pyg_data
from torch_geometric.nn import GCNConv
from sklearn.neighbors import NearestNeighbors
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GATConv
import networkx as nx
from sklearn.decomposition import PCA
import scipy.io
import sys
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import time

In [77]:
### Hyperparameters
run_epochs = 1000
lr = 0.01
weight_decay = 5e-4
graph_type = 'knn_drop'      # 'knn_drop', 'knn_nodrop', 'random1', 'random2'
k_neighbors = 30
knn_metric = 'cosine'      # 'cosine', 'p', 'sqeucledian'
drop_classes = 0
drop_edges = k_neighbors*(k_neighbors-1)//2
drop_ratio = 1
# 'seuclidean', 'p', 'sqeuclidean', 'mahalanobis', 'pyfunc', 'jaccard', 'nan_euclidean', 'cityblock', 'manhattan', 'precomputed', 'cosine', 'yule', 'infinity', 'sokalsneath', 'rogerstanimoto', 'euclidean', 'russellrao', 'canberra', 'haversine', 'correlation', 'l1', 'chebyshev', 'sokalmichener', 'braycurtis', 'dice', 'l2', 'hamming', 'minkowski'
density = 0.1

In [78]:
file_paths = ['dataset/ADNI.mat', 'dataset/PPMI.mat', 'dataset/ADNI_fMRI.mat', 'dataset/FTD_fMRI.mat', 'dataset/OCD_fMRI.mat']
methods_acc = []
methods_std = []
# file_paths = ['dataset/ADNI_90_120_fMRI.mat', 'dataset/FTD_90_200_fMRI.mat', 'dataset/OCD_90_200_fMRI.mat']

for file_path in file_paths:

    seeds=[0, 1, 2, 3, 4]
    seeds_best_acc = []

    for seed in seeds:

        # 设置随机种子
        def set_seed(seed):
            torch.manual_seed(seed)
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            np.random.seed(seed)
            random.seed(seed)
            torch.manual_seed(seed)
            torch.backends.cudnn.benchmark = False
            torch.backends.cudnn.deterministic = True
            print('Random seed is set to {}.'.format(seed))
        set_seed(seed)


        mat_data=scipy.io.loadmat(file_path)

        train_ratio=0.6
        valid_ratio=0.2
        test_ratio=0.2

        X_train=[]
        X_test=[]
        y_train=[]
        y_test=[]
        X_valid=[]
        y_valid=[]

        y_train_master=[] # 在主分类器中, 将 MCI, MCIn, MCIp 合并为一类的训练集标签
        y_valid_master=[]
        y_test_master=[]
        Labels=[key for key in mat_data.keys() if not key.startswith('__')]
        dict_labels = {label: i for i, label in enumerate(Labels)}
        print(dict_labels)
        scaler=StandardScaler()

        X_train_MCI=[] # MCI, MCIn, MCIp 的训练集, 验证集及其标签
        X_valid_MCI=[]
        y_train_MCI=[]
        y_valid_MCI=[]
        X_test_MCI=[]
        y_test_MCI=[]


        for label, data in mat_data.items():
            if label.startswith('__'):
                continue
            label_num = dict_labels[label]
            N = np.shape(data)[0]
            indices = np.random.permutation(N)
            train_end = int(train_ratio * N)
            valid_end = int((train_ratio + valid_ratio) * N)

            train_index = indices[:train_end]
            validation_index = indices[train_end:valid_end]
            test_index = indices[valid_end:]
            # print(indices)
            # print(train_index,validation_index,test_index)
            for i in range(N):
                if i in train_index:
                    X_train.append(data[i])
                    y_train.append(label_num)
                    if file_path == 'dataset/ADNI.mat' and label_num>=1 and label_num<=3:
                        y_train_master.append(-1)
                        X_train_MCI.append(data[i]) # MCI, MCIn, MCIp 的训练集
                        y_train_MCI.append(label_num)
                    else:
                        y_train_master.append(label_num)
                if i in validation_index:
                    X_valid.append(data[i])
                    y_valid.append(label_num)
                    if file_path == 'dataset/ADNI.mat' and label_num>=1 and label_num<=3:
                        y_valid_master.append(-1)
                        X_valid_MCI.append(data[i]) # MCI, MCIn, MCIp 的验证集
                        y_valid_MCI.append(label_num)
                    else:
                        y_valid_master.append(label_num)
                if i in test_index:
                    X_test.append(data[i])
                    y_test.append(label_num)
                    if file_path == 'dataset/ADNI.mat' and label_num>=1 and label_num<=3:
                        y_test_master.append(-1)
                        X_test_MCI.append(data[i]) # MCI, MCIn, MCIp 的测试集
                        y_test_MCI.append(label_num)
                    else:
                        y_test_master.append(label_num)
                    

        dict_labels_MCI = {label: i for i, label in enumerate(Labels[1:])}
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        y_train_master = np.array(y_train_master)

        X_valid = np.array(X_valid)
        y_valid = np.array(y_valid)
        y_valid_master = np.array(y_valid_master)

        X_train_MCI = np.array(X_train_MCI)
        y_train_MCI = np.array(y_train_MCI)

        X_valid_MCI = np.array(X_valid_MCI)
        y_valid_MCI = np.array(y_valid_MCI)

        X_test = np.array(X_test)
        y_test = np.array(y_test)
        y_test_master = np.array(y_test_master)
        def normalize(X):
            if len(X) != 0:
                scaler = StandardScaler()
                X = scaler.fit_transform(X)
                # X=(X-np.mean(X,axis=0))/np.std(X,axis=0)
            return X



        if file_path in ['dataset/ADNI.mat', 'dataset/PPMI.mat']:
            X_train = normalize(X_train)
            X_valid = normalize(X_valid)
            X_train_MCI = normalize(X_train_MCI)
            X_valid_MCI = normalize(X_valid_MCI)
            X_test = normalize(X_test)
            X_test_MCI = normalize(X_test_MCI)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')









        ### Load data
        train_num = X_train.shape[0]
        val_num = X_valid.shape[0]
        test_num = X_test.shape[0]
        if file_path in ['dataset/ADNI.mat', 'dataset/PPMI.mat']:
            num_features = X_train.shape[1]
        else:
            num_features = X_train.shape[1]*X_train.shape[2]
        num_classes = len(Labels)


        ## Construct the graph
        def construct_knn_drop_graph(X, y, val_idx, k=5, metric='unsupervised', ratio=0.3, drop_classes=1, density=0.1, drop_edges=24):
            '''
            Description: Construct the knn graph of the data.
            Input:
            - X: Train data and test data
            - y: The label of train data.
            - k: The number of nearest neighbors.
            Return:
                - the knn graph of the data.
            '''
            knn = NearestNeighbors(n_neighbors=k, metric=metric)
            knn.fit(X)
            _, indices = knn.kneighbors(X) # Indices of the nearest points in the population matrix.
            adj_mat = np.zeros((len(X), len(X)))
            for i in range(len(X)):
                for j in range(k):
                    if i>=val_idx[0]:
                        drop_class_idx = random.sample(range(len(Labels)), drop_classes)
                        if indices[i][j] < val_idx[0] and y[indices[i][j]] in drop_class_idx:
                            continue
                    adj_mat[i][indices[i][j]] = 1
                    adj_mat[indices[i][j]][i] = 1

            dist_valid=[]
            dist_test=[]
            for i in range(len(X)):  # 尝试一些删边手段，修改邻接矩阵
                for j in range(i+1,len(X)):
                    if i >= val_idx[0] and i<test_idx[0] and j >= val_idx[0] and j<test_idx[0]: # 验证集内部进行删边
                        if adj_mat[i][j]==1:
                            # print(np.linalg.norm(np.squeeze(X[i])-np.squeeze(X[j])))
                            dist_valid.append([np.linalg.norm(X[i]-X[j]),[i,j]])
                    if i>=test_idx[0] and j>=test_idx[0]: # 测试集内部进行删边
                        if adj_mat[i][j]==1:
                            dist_test.append([np.linalg.norm(X[i]-X[j]),[i,j]])
                        # 测试集内部进行删边
                        # drop_edge_idx_test = random.sample(range(val_idx[0],test_idx[0]), drop_edges)
                        # if indices[i][j] >= test_idx[0] and y[indices[i][j]] in drop_edge_idx_test:
                        #     continue

                        # 随机选择 drop_class 个类，断开测试集和验证集中的样本与训练集中这些类的连边
                    

                        # 随机选择 k/2 个邻居连边
                        # random_sample = random.sample(range(k), int(k*ratio))
                        # if j in random_sample:
                        #     adj_mat[i][indices[i][j]] = 1
                        #     adj_mat[indices[i][j]][i] = 1
            dist_valid.sort(key=lambda x:x[0],reverse=True)
            dist_test.sort(key=lambda x:x[0],reverse=True) # 按照距离进行排序
            dist_valid=np.array(dist_valid)
            dist_test=np.array(dist_test)
            drop_valid=dist_valid[:drop_edges]
            drop_test=dist_test[:drop_edges]
            for _,idx in drop_valid:
                adj_mat[idx[0]][idx[1]]=0
                adj_mat[idx[1]][idx[0]]=0
            for _,idx in drop_test:
                adj_mat[idx[0]][idx[1]]=0
                adj_mat[idx[1]][idx[0]]=0
            for i in val_idx:
                for j in test_idx:
                    adj_mat[i][j]=0
                    adj_mat[j][i]=0
            # 将连边加入邻接矩阵
            edge_index = []
            for i in range(len(X)):
                for j in range(i+1,len(X)):
                    # 随机断开验证集和测试集内的点与连边
                    if adj_mat[i][j]==1: # 对于训练集中的数据，只有两个是同一类才连边
                        # edge_index.append([i,j])
                        if i>=train_num or j>=train_num: # 如果两个点有一个不是训练集，就直接根据邻接矩阵连边
                            edge_index.append([i, j])
                        if i<train_num and j<train_num and y[i]==y[j]:
                            edge_index.append([i, j])
                        # if i >= val_idx[0] and np.random.rand()<=density: # 对于验证集内部的点，随机断开一些连边
                        # drop_edge_idx = random.sample(range(val_idx[0],test_idx[0]), drop_edges)
                        # if 
            edge_index = torch.tensor(edge_index).T
            edge_index = edge_index.to(device)
            X = X.astype(np.float32)
            X = torch.tensor(X).float().to(device)
            y = torch.tensor(y).long().to(device)
            # print(X.shape,y.shape)
            data = pyg.data.Data(x=X, edge_index=edge_index, y=y)
            return data


        def construct_knn_nodrop_graph(X, y, val_idx, k=5, metric='unsupervised', ratio=0.3, drop_class=0, density=0.1, drop_edges=0):
            '''
            Description: Construct the knn graph of the data.
            
            Input:
            - X: Train data and test data.
            - y: The label of train data.
            - k: The number of nearest neighbors.
            
            Return:
                - the knn graph of the data.
            '''
            knn=NearestNeighbors(n_neighbors=k, metric=metric)
            knn.fit(X)
            _, indices = knn.kneighbors(X) # Indices of the nearest points in the population matrix.
            adj_mat=np.zeros((len(X),len(X)))
            for i in range(len(X)):
                for j in range(k):
                    adj_mat[i][indices[i][j]]=1
            edge_index = []
            for i in range(len(X_train)):
                for j in range(len(X_train)):
                    if j>i and adj_mat[i][j]==1:
                        edge_index.append([i,j])
            edge_index = torch.tensor(edge_index).T
            edge_index = edge_index.to(device)
            X=X.astype(np.float32)
            X = torch.tensor(X).float().to(device)
            y = torch.tensor(y).long().to(device)
            # print(X.shape,y.shape)
            data = pyg.data.Data(x=X,edge_index=edge_index,y=y)
            return data



        def construct_random1_graph(X, y, num_neighbors=5, density=0.1):
            '''
            Description: Construct a random graph from the data.
            Input:
            - X: Data.
            - y: Labels.
            - num_neighbors: The number of neighbors for each node.
            - density: The edge density in the random graph.
            Return:
            - A random graph as a PyTorch Geometric Data object.
            '''

            num_nodes = len(X)
            adj_mat = np.zeros((num_nodes, num_nodes))

            for i in range(num_nodes):
                for j in range(i + 1, num_nodes):
                    if np.random.rand() < density:
                        adj_mat[i][j] = 1
                        adj_mat[j][i] = 1

            # Randomly select 'num_neighbors' neighbors for each node
            for i in range(num_nodes):
                neighbors = np.where(adj_mat[i] == 1)[0]
                if len(neighbors) > num_neighbors:
                    random_neighbors = np.random.choice(neighbors, num_neighbors, replace=False)
                    adj_mat[i] = 0
                    adj_mat[i][random_neighbors] = 1

            edge_index = np.where(adj_mat == 1)
            edge_index = torch.tensor(edge_index, dtype=torch.long).to(device)

            X = X.astype(np.float32)
            X = torch.tensor(X, dtype=torch.float32).to(device)
            y = torch.tensor(y, dtype=torch.long).to(device)

            data = pyg.data.Data(x=X, edge_index=edge_index, y=y)

            return data


        def construct_random2_graph(X, y, num_neighbors=5, prob_rewire=0.2):
            '''
            Description: Construct a random graph from the data.
            Input:
            - X: Data.
            - y: Labels.
            - num_neighbors: The number of neighbors for each node.
            - prob_rewire: The probability of rewiring edges in the Watts-Strogatz model.
            Return:
            - A random graph as a PyTorch Geometric Data object.
            '''

            # Create a Watts-Strogatz random graph
            G = nx.watts_strogatz_graph(len(X), num_neighbors, prob_rewire)

            # Convert the NetworkX graph to a PyTorch Geometric Data object
            edge_index = torch.tensor(list(G.edges()), dtype=torch.long).t()
            edge_index = edge_index.to(device)

            X = X.astype(np.float32)
            X = torch.tensor(X, dtype=torch.float32).to(device)
            y = torch.tensor(y, dtype=torch.long).to(device)

            data = pyg_data.Data(x=X, edge_index=edge_index, y=y)

            return data

# class FeedbackModel(nn.Module):
#     def __init__(self, embedding_matrix):
        
#         super(FeedbackModel, self).__init__()

#         self.embed = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=False)
#         # GCNConv SAGEConv ResGatedGraphConv GraphConv(300, 128) 
#         # TransformerConv GATv2Conv GATConv(300, 128, heads=4) ChebConv(300, 128, K=2)
#         # GCNConv SAGEConv ResGatedGraphConv GraphConv(128, 64) 
#         # TransformerConv  GATv2Conv GATConv(4*128, 64) ChebConv(128, 64, K=2)
# #         self.gru = nn.GRU(256, 256, num_layers=1, 
# #                           dropout=0, batch_first=True,
# #                           bidirectional=False)          # RNN, GRU
#         # output: (N, L, D∗Hout), D = 2 if bidirectional=True otherwise 1
#         # h_n: (D∗num_layers, N, Hout)
#         self.gc1   = GATv2Conv(300, 128)
#         self.pool1 = pyg_nn.TopKPooling(128, ratio=0.8)
#         self.gc2   = GCNConv(128, 128)
#         self.pool2 = pyg_nn.TopKPooling(128, ratio=0.8)
#         self.lin1  = nn.Linear(256, 64)
#         self.lin2  = nn.Linear(64, 6)

#     def forward(self, data):
#         x, edge_index, batch = data.x, data.edge_index, data.batch
#         x = x.squeeze(1)
#         x = self.embed(x)
        
#         x = F.relu(self.gc1(x, edge_index))
#         x, edge_index, edge_attr, batch, perm, score = self.pool1(x, edge_index, None, batch)
#         x1 = torch.cat([pyg_nn.global_max_pool(x, batch), pyg_nn.global_mean_pool(x, batch)], dim=1)

#         x = F.relu(self.gc2(x, edge_index))
#         x, edge_index, edge_attr, batch, perm, score = self.pool2(x, edge_index, None, batch)
#         x2 = torch.cat([pyg_nn.global_max_pool(x, batch), pyg_nn.global_mean_pool(x, batch)], dim=1)

#         x = x1 + x2
#         # x, hn = self.gru(x, None)
#         x = F.relu(self.lin1(x))
#         x = F.dropout(x, p=0.5, training=self.training)
#         output = F.relu(self.lin2(x))

#         return output


        ### Define the model
        class GCN(nn.Module):
            def __init__(self, num_node_features, num_classes):
                super().__init__()

                self.num_classes = num_classes

                # self.conv1 = GCNConv(num_node_features, 64)
                # self.conv2 = GCNConv(64, num_classes)
                # self.norm = torch.nn.BatchNorm1d(64)
                self.conv1 = GATConv(num_node_features, 4, heads=8, dropout=0.5)
                self.conv2 = GATConv(4*8, num_classes, heads=8, concat=False, dropout=0.5)
                self.norm = torch.nn.BatchNorm1d(4*8)

            def forward(self, data):
                x, edge_index = data.x, data.edge_index

                x = self.conv1(x, edge_index)
                x = self.norm(x)
                x = F.relu(x)
                # x = F.dropout(x, training=self.training)
                x = self.conv2(x, edge_index)

                return x


        ### Train the model
        def train(model):

            train_acc, val_acc, test_acc = 0, 0, 0
            best_train_acc, best_val_acc, best_test_acc = 0, 0, 0
            best_test_epoch = 0
            avg_test_acc = 0

            # track accuracies over epochs
            train_accuracies = []
            val_accuracies = []
            test_accuracies = []

            optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
            loss_function = torch.nn.CrossEntropyLoss().to(device)
            for epoch in range(run_epochs):
                model.train()
                out = model(data)
                optimizer.zero_grad()
                # print(data.train_mask.shape, data.train_mask.shape)
                # print(out.shape,data.y.shape)
                # print(data.y[data.train_mask].shape)
                # print(out[data.train_mask].shape)
                loss = loss_function(out[data.train_mask], data.y[data.train_mask])
                loss.backward()
                optimizer.step()

                train_acc, val_acc, test_acc = test()
                train_accuracies.append(train_acc)
                val_accuracies.append(val_acc)
                test_accuracies.append(test_acc)
                if test_acc > best_test_acc:
                    best_train_acc = train_acc
                    best_val_acc = val_acc
                    best_test_acc = test_acc
                    best_test_epoch = epoch


            avg_test_acc = sum(test_accuracies) / run_epochs
            seeds_best_acc.append(best_test_acc)

            # print accuracies of best epoch and the following 9 epochs
            for i in range(best_test_epoch, min(run_epochs, best_test_epoch + 10)):
                print(f'Epoch {i+1}: Train: {train_accuracies[i]:.4f}, Val: {val_accuracies[i]:.4f}, Test: {test_accuracies[i]:.4f}')

            print(f"Best test epoch {best_test_epoch+1}: train_acc={best_train_acc:.4f}, val_acc={best_val_acc:.4f}, test_acc={best_test_acc:.4f}")
            print(f"avg_acc: {avg_test_acc:.4f}")

            plt_epoch = min(best_test_epoch+9, run_epochs)

            # plot from first epoch to the best epoch
            plt.plot(range(1, plt_epoch+1), train_accuracies[:plt_epoch], label='Train Accuracy')
            plt.plot(range(1, plt_epoch+1), val_accuracies[:plt_epoch], label='Validation Accuracy')
            plt.plot(range(1, plt_epoch+1), test_accuracies[:plt_epoch], label='Test Accuracy')
            plt.ylim([min(train_accuracies[:plt_epoch]+val_accuracies[:plt_epoch]+test_accuracies[:plt_epoch]), 
                    max(train_accuracies[:plt_epoch]+val_accuracies[:plt_epoch]+test_accuracies[:plt_epoch])]) 
            # plt.plot(range(1, run_epochs+1), train_accuracies[:], label='Train Accuracy')
            # plt.plot(range(1, run_epochs+1), val_accuracies[:], label='Validation Accuracy')
            # plt.plot(range(1, run_epochs+1), test_accuracies[:], label='Test Accuracy')
            # plt.ylim([min(train_accuracies[:]+val_accuracies[:]+test_accuracies[:]), 
            #         max(train_accuracies[:]+val_accuracies[:]+test_accuracies[:])]) 
            plt.xlabel('Epochs')
            plt.ylabel('Accuracy')
            plt.legend()
            plt.show()



        ### Test the model
        def test():
            model.eval()
            log, accs = model(data), []
            for _, mask in data('train_mask', 'val_mask', 'test_mask'):
                pred = log[mask].max(1)[1]
                acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
                accs.append(acc)
            return accs


        model = GCN(num_features, num_classes).to(device)

        ### Load and split the data
        if file_path in ['dataset/ADNI.mat', 'dataset/PPMI.mat']:
            X = np.concatenate((X_train, X_valid, X_test), axis=0)
            y = np.concatenate((y_train, y_valid, y_test), axis=0)
        else:
            X_train = X_train.reshape(X_train.shape[0], -1)
            X_valid = X_valid.reshape(X_valid.shape[0], -1)
            X_test = X_test.reshape(X_test.shape[0], -1)
            X = np.concatenate((X_train, X_valid, X_test), axis=0)
            y = np.concatenate((y_train, y_valid, y_test), axis=0)

        train_idx = np.array(range(X_train.shape[0]))
        val_idx = np.array(range(X_train.shape[0], X_train.shape[0]+X_valid.shape[0]))
        train_and_val_idx = np.array(range(X_train.shape[0]+X_valid.shape[0]))
        test_idx = np.array(range(X_train.shape[0]+X_valid.shape[0], X_train.shape[0]+X_valid.shape[0]+X_test.shape[0]))
        print("Shape of data matrix: ", X.shape)
        print("Shape of labels vector: ", y.shape)
        all_f = np.zeros((X.shape[0],), dtype=np.bool_)

        if graph_type == 'knn_drop':
            data = construct_knn_drop_graph(X, y, val_idx=val_idx, k=k_neighbors, metric=knn_metric, ratio=drop_ratio, drop_classes=drop_classes, density=density, drop_edges=drop_edges)
        elif graph_type == 'knn_nodrop':
            data = construct_knn_nodrop_graph(X, y, val_idx=val_idx, k=k_neighbors, metric=knn_metric, ratio=drop_ratio, density=density)
        elif graph_type == 'random1':
            data = construct_random1_graph(X, y, val_idx=val_idx, num_neighbors=k_neighbors, density=density)
        elif graph_type == 'random2':
            data = construct_random2_graph(X, y, val_idx=val_idx, num_neighbors=k_neighbors, prob_rewire=density)

        all_f_tmp = all_f.copy()
        all_f_tmp[train_idx] = True
        train_mask = all_f_tmp

        all_f_tmp = all_f.copy()
        all_f_tmp[val_idx] = True
        val_mask = all_f_tmp

        all_f_tmp = all_f.copy()
        all_f_tmp[train_and_val_idx] = True
        train_and_val_mask = all_f_tmp

        all_f_tmp = all_f.copy()
        all_f_tmp[test_idx] = True
        test_mask = all_f_tmp

        # print(y.shape, train_idx.shape, val_idx.shape, train_and_val_idx.shape, test_idx.shape)
        # print(train_mask.shape, val_mask.shape, train_and_val_mask.shape, test_mask.shape)
        # print(y[train_mask].shape, y[val_mask].shape, y[train_and_val_mask].shape, y[test_mask].shape)

        data.train_mask = train_mask
        data.val_mask = val_mask
        data.train_and_val_mask = train_and_val_mask
        data.test_mask = test_mask

        ### Train the model and test the model
        # train()
        # ('', <class '__main__.GCN'>)
        # ('conv1', <class 'torch_geometric.nn.conv.gat_conv.GATConv'>)
        # ('conv1.aggr_module', <class 'torch_geometric.nn.aggr.basic.SumAggregation'>)
        # ('conv1.lin_src', <class 'torch_geometric.nn.dense.linear.Linear'>)
        # ('conv2', <class 'torch_geometric.nn.conv.gat_conv.GATConv'>)
        # ('conv2.aggr_module', <class 'torch_geometric.nn.aggr.basic.SumAggregation'>)
        # ('conv2.lin_src', <class 'torch_geometric.nn.dense.linear.Linear'>)
        # ('norm', <class 'torch.nn.modules.batchnorm.BatchNorm1d'>)
        # for x in [(n,type(m))for n,m in model.named_modules()]:
        #     print(x)
        import peft
        from peft import LoraConfig
        peft_config = LoraConfig(
            target_modules=['conv1','conv1.lin_src', 'conv2'],
            modules_to_save=['conv2.lin_src'],
        )
        import copy

        module = GCN(num_features, num_classes).to(device)
        module_copy = copy.deepcopy(module)  # we keep a copy of the original model for later
        peft_model = peft.get_peft_model(module, peft_config)
        optimizer = torch.optim.Adam(peft_model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()
        peft_model.print_trainable_parameters()
        
        %time train(peft_model)

for metric in ['cosine', 'p', 'sqeuclidean', 'mahalanobis', 'pyfunc', 'jaccard', 'nan_euclidean', 'cityblock', 'manhattan', 'precomputed', 'yule', 'infinity', 'sokalsneath', 'rogerstanimoto', 'euclidean', 'russellrao', 'canberra', 'haversine', 'correlation', 'l1', 'chebyshev', 'sokalmichener', 'braycurtis', 'dice', 'l2', 'hamming', 'minkowski']:
                


    seeds_avg_acc = sum(seeds_best_acc) / 5

    print(f"{file_path}: The average accuracy is {seeds_avg_acc:.4f}")
    methods_acc.append(seeds_avg_acc)
    methods_std.append(np.std(seeds_best_acc))
    print("---------------------------------------------------------------------------------------------------------------------------------\n")



print("#########################################################################################################################################\n")
i = 0
for file_path in file_paths:
    print(f"{file_path}: The average accuracy is {methods_acc[i]:.4f}, std is {methods_std[i]:.4f}")
    i = i + 1

Random seed is set to 0.
{'AD': 0, 'MCI': 1, 'MCIn': 2, 'MCIp': 3, 'NC': 4}
Shape of data matrix:  (301, 186)
Shape of labels vector:  (301,)


C:\Users\Calculus\AppData\Local\Temp\ipykernel_9000\2039538187.py:206: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dist_valid=np.array(dist_valid)
C:\Users\Calculus\AppData\Local\Temp\ipykernel_9000\2039538187.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dist_test=np.array(dist_test)


ValueError: Target module GATConv(186, 4, heads=8) is not supported. Currently, only the following modules are supported: `torch.nn.Linear`, `torch.nn.Embedding`, `torch.nn.Conv2d`, `transformers.pytorch_utils.Conv1D`.